# 数据清洗、信息提取

## 初始化

In [33]:
import pandas as pd
import time

amount = pd.read_csv('data2.csv',encoding='gb2312')
student = pd.read_csv('data1.csv',encoding='gb2312')

# 日期分割成两个字段
amount['Time'] = amount['Date'].apply(lambda date: time.strftime('%H:%M',(time.strptime(date,'%Y/%m/%d %H:%M'))))
amount['Date'] = amount['Date'].apply(lambda date: date.split(' ')[0])

## 对data2进行数据清洗

In [34]:
# 去除重复数据项
amount.drop_duplicates(subset=['CardNo','CardCount'])

# TerSerNo非空的数据为0点结算，作为异常数据排除
amount = amount[amount.TermSerNo.isnull()]
# conOperNo非空的数据是非消费数据，作为不需要的数据删除
amount = amount[amount.conOperNo.isnull()]

# 以防万一，再加一条，显示删除非消费数据
amount = amount[amount.Type=='消费']

# 删除这些被使用过，已经不再需要的字段
del amount['TermSerNo']
del amount['conOperNo']
del amount['FundMoney']
del amount['Type']

# 删除其他无效的数据
del amount['Index']
del amount['PeoNo']

# data1的数据清洗
del student['Index']

## 消费数据整合

整合成两张表

- consume.csv, 所有的有用的消费数据，但缺少年级、性别等等持卡人信息。

- grade18.csv, 18级学生的消费数据，包含持卡人信息。

In [35]:
# 保存consume.csv
amount.to_csv('consume.csv')

# 合并data1和data2数据集
data = pd.merge(student, amount, on=['CardNo'])
# 保存grade18.csv
data.to_csv('grade18.csv')